<a href="https://colab.research.google.com/github/matthewreed2000/cse380-notebooks/blob/master/09_2_Ponder_and_Prove_Data_Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ponder and Prove Data Compression
#### Due: Saturday, 6 March 2021, 11:59 pm.

*Matthew Reed*

In collaboration with:
- Davis Kerr
- Brayden Whitlock
- Paul Semadeni

# Explore Huffman Trees and Huffman Codes


Your task is examine how to compress a *special piece of information* as compactly as possible, and **calculate various compression ratios**.

Recall that the **compression ratio** of a variable-length encoding like Huffman encoding is the percentage $100(f - v)/f$, where $f$ is the number of bits per symbol of the smallest **fixed**-length encoding, and $v$ is the average number of bits per symbol with the variable-length encoding.

For example, if there were 9 different symbols in a message, $f=4$ is the number of bits of the smallest fixed-length encoding, because $2^3 = 8$ (not enough for $9$) and $2^4 = 16$ (enough and to spare). If the variable-length encoding of the message had $v=3.12$, the compression ratio would be $100(4 - 3.12)/4 \approx 22\%$.

Note that calculating the average number of bits per symbol is not strictly necessary. That's because an alternate and equivalent way is to calculate $100(ft - vt)/ft$, where $ft$ is the **total** number of bits encoded with the fixed encoding, and $vt$ is the **total** number of bits encoded with the variable-length encoding.

The *special piece of information* to be compressed is a list of the first ten million primes. This is a list that starts

|    |
|----|
|  2 |
|  3 |
|  5 |
|  7 |
| 11 |
| 13 |
| 17 |
| 19 |
| 23 |
| 29 |

  and ends

|           |
|-----------|
| 179424551 |
| 179424571 |
| 179424577 |
| 179424601 |
| 179424611 |
| 179424617 |
| 179424629 |
| 179424667 |
| 179424671 |
| 179424673 |

As ASCII text stored in a file with one prime per line, the size of this data file is slightly over 89 megabytes. The goal is to compress this down to just over 5 megabytes (5589056 bytes, to be exact). That's a 94% compression ratio!

Standard compression tools can only get about a 73% compression ratio for this ASCII data. A more clever approach is needed. Instead of compressing the list of prime numbers, compress a list of the *gaps* between them!

It doesn't save much, just the unique (occurring only once) gap size of 1 between 2 and 3, but in the spirit of de Polignac's conjecture that every *even* number appears infinitely often as a gap between consecutive primes, just consider the even-sized gaps. The result will be a list that starts with 2 (the difference between 5 and 3), 2 (the difference between 7 and 5), 4 (the difference between 11 and 7), 2 (the difference between 13 and 11), 4 (the difference between 17 and 13), 2 (the difference between 19 and 17), 4 (the difference between 23 and 19), and 6 (the difference between 29 and 23).

Generating this data is the first task. The algorithm for doing so is very straightforward:

1. Find the gaps between consecutive odd primes.
2. Store these gaps as a list of even numbers.

Tabulating the results, the first ten gaps and the last ten gaps are as follows, where the numbers after the equals signs are the gaps to list:

|                 |
|-----------------|
|  5  -   3  =  2 |
|  7  -   5  =  2 |
| 11  -   7  =  4 |
| 13  -  11  =  2 |
| 17  -  13  =  4 |
| 19  -  17  =  2 |
| 23  -  19  =  4 |
| 29  -  23  =  6 |
| 31  -  29  =  2 |
| 37  -  31  =  6 |

|                                |
|--------------------------------|
| 179424551  -  179424533  =  18 |
| 179424571  -  179424551  =  20 |
| 179424577  -  179424571  =   6 |
| 179424601  -  179424577  =  24 |
| 179424611  -  179424601  =  10 |
| 179424617  -  179424611  =   6 |
| 179424629  -  179424617  =  12 |
| 179424667  -  179424629  =  38 |
| 179424671  -  179424667  =   4 |
| 179424673  -  179424671  =   2 |

As a correctness check, see if your generated list of gaps has length 9999998.

The next step is to count how many times each gap size occurs, so that for the Huffman encoding scheme, the larger the frequency of occurrence, the smaller the number of bits encoding that gap size.

As a correctness check, here are the first ten and the last ten gap counts:

|  Gap | Count   |
|------|---------|
|    2 |  738597 |
|    4 |  738717 |
|    6 | 1297540 |
|    8 |  566151 |
|   10 |  729808 |
|   12 |  920661 |
|   14 |  503524 |
|   16 |  371677 |
|   18 |  667734 |
|   20 |  354267 |
|      |         |
|  190 |       1 |
|  192 |       3 |
|  194 |       1 |
|  196 |       1 |
|  198 |       6 |
|  202 |       2 |
|  204 |       3 |
|  210 |       4 |
|  220 |       1 |
|  222 |       1 |

Note two things from these partial gap counts:

1. Small even numbers (< 100) are well represented, larger ones (< 1000) less so.
2. Ten million primes aren't enough to have *every* even number represented; for example, 200, 206, 208, 212, 214, 216, and 218 do not appear even once.


## Generate Gaps

In [1]:
!pip install pyprimesieve

  Created wheel for pyprimesieve: filename=pyprimesieve-0.1.6-cp37-cp37m-linux_x86_64.whl size=368951 sha256=f18f17250124da611621d78d6a8f662613c99eee4b893799c994c1e8fdbf91ac
  Stored in directory: /root/.cache/pip/wheels/c4/63/2b/a485079de882a375d28a4dc141386c76ea9a6aaad505f2198b
Successfully built pyprimesieve


In [2]:
# Provided by Davis
from pyprimesieve import primes, primes_nth
def prime_gaps(range):
  list_of_gaps = []
  prev = 3
  gap = 0
  for prime in list(primes(4, primes_nth(range + 1))):
    gap = prime - prev
    prev = prime
    list_of_gaps.append(gap)
  return list_of_gaps

In [3]:
gaps = prime_gaps(10000000)
print(len(gaps))
print(gaps[:10])
print(gaps[-10:])

9999998
[2, 2, 4, 2, 4, 2, 4, 6, 2, 6]
[18, 20, 6, 24, 10, 6, 12, 38, 4, 2]


(I count the number of gaps in a later section when generating the Huffman tree)

# Determine Exact Size of Data to be Compressed


Without actually doing it, imagine creating an ASCII file containing the first ten million primes, represented in decimal, one prime per line. Calculate the size of this file, so you can show an exceptional compression ratio from it (see below).

Using a binary encoding instead of ASCII, each prime requires 32 bits (4 bytes), so the size of a binary file is easily determined.

Using a fixed-width encoding of the gap counts, however, requires knowing how many different gap sizes there are, after which the calculation is straightforward.

## Finding the Sizes

In [4]:
raw_primes = [*primes(1, primes_nth(10000000 + 1))]
ascii_bytes = sum([len(str(x)) for x in raw_primes])
ascii_bytes += (len(raw_primes) - 1) # Account for \n characters

print(f'Encoded as ASCII, the list of gaps would take up {ascii_bytes} bytes or {ascii_bytes/(2**20):0.2f} MB.')

Encoded as ASCII, the list of gaps would take up 93484449 bytes or 89.15 MB.


In [5]:
prime_fixed_bytes = len(raw_primes) * 4
print(f"If the raw primes are stored in a binary file, the file would be {prime_fixed_bytes} bytes or {prime_fixed_bytes / (2**20):0.3f} MB.")

If the raw primes are stored in a binary file, the file would be 40000000 bytes or 38.147 MB.


In [6]:
len(set(gaps))

104

For fixed-length encoding, each prime would need 7 bits (since $2^6 \le 104 < 2^7$). Thus, the length of a binary file of the gaps between primes using fixed-length encoding would come out to

In [7]:
gap_bytes = len(gaps) * 7 / 8
print(f"{gap_bytes} bytes or {gap_bytes / (2**20):0.3f} MB.")

8749998.25 bytes or 8.345 MB.


# Use Functional Python


You are encouraged to use the [anytree](https://pypi.org/project/anytree) Python library, which has a nice exporter by way of which you can graphically view trees. (You may recall using this in DM1, and thus know that **anytree** depends on [graphviz](https://graphviz.org), which you also used.)

This library uses the object-oriented features of Python to create and visualize trees. You are encouraged to use the functional features of Python as much as possible, achieving your results not by using some existing third-party libraries for building Huffman Trees and Codes, but writing your own code as cleanly and elegantly as you can.

## Implementation

In [8]:
# Implement the Ever-Shrinking Queue
def es_queue(l):
  l.sort(key=lambda x: x[0])
  if len(l) == 1:
    return l[0]
  else:
    return es_queue([[l[0][0] + l[1][0], [l[0],l[1]]]] + l[2:])

# Convert the tree from es_queue() into a list of tuples
def get_tuples(l, path=''):
  if type(l[1]) is list:
    tuple_right = get_tuples(l[1][1], path+'1')
    tuple_left = get_tuples(l[1][0], path+'0')
    return tuple_left + tuple_right
  else:
    return [(l[1], path)]

In [9]:
from collections import Counter
gap_count = Counter(gaps)
gap_count = [[v, k] for k,v in gap_count.items()]
gap_count

[[738597, 2],
 [738717, 4],
 [1297540, 6],
 [566151, 8],
 [503524, 14],
 [729808, 10],
 [920661, 12],
 [667734, 18],
 [354267, 20],
 [307230, 22],
 [129043, 34],
 [453215, 24],
 [371677, 16],
 [211203, 26],
 [229177, 28],
 [398713, 30],
 [123123, 32],
 [206722, 36],
 [64866, 44],
 [159956, 42],
 [111546, 40],
 [38800, 52],
 [93693, 48],
 [94682, 38],
 [17255, 72],
 [52183, 50],
 [16763, 62],
 [64157, 54],
 [55305, 60],
 [27985, 58],
 [54931, 46],
 [32224, 56],
 [17374, 64],
 [12368, 68],
 [3411, 86],
 [30960, 66],
 [17475, 70],
 [13758, 78],
 [7253, 76],
 [4791, 82],
 [3544, 96],
 [711, 112],
 [1923, 100],
 [8540, 74],
 [7056, 90],
 [9818, 84],
 [1125, 114],
 [6760, 80],
 [3454, 88],
 [1831, 98],
 [2259, 92],
 [933, 106],
 [2058, 94],
 [433, 118],
 [301, 132],
 [1168, 104],
 [2374, 102],
 [941, 110],
 [533, 126],
 [948, 120],
 [67, 148],
 [1634, 108],
 [287, 122],
 [210, 138],
 [183, 128],
 [43, 154],
 [211, 130],
 [439, 116],
 [46, 146],
 [100, 136],
 [318, 124],
 [128, 134],
 [140, 1

In [10]:
huff_tuples = get_tuples(es_queue(gap_count))
huff_dict = {x[0]:x[1] for x in huff_tuples}
huff_dict

{2: '1100',
 4: '1101',
 6: '100',
 8: '0101',
 10: '1011',
 12: '000',
 14: '0100',
 16: '11100',
 18: '0111',
 20: '10101',
 22: '01101',
 24: '0010',
 26: '111110',
 28: '111111',
 30: '11101',
 32: '001110',
 34: '011000',
 36: '111101',
 38: '1010011',
 40: '001100',
 42: '101000',
 44: '0011111',
 46: '11110010',
 48: '1010010',
 50: '11110001',
 52: '01100111',
 54: '0011110',
 56: '01100100',
 58: '00110101',
 60: '11110011',
 62: '011001010',
 64: '011001100',
 66: '00110110',
 68: '1111000010',
 70: '011001101',
 72: '011001011',
 74: '0011011111',
 76: '0011011101',
 78: '001101000',
 80: '11110000111',
 82: '11110000001',
 84: '1111000001',
 86: '00110100100',
 88: '00110100101',
 90: '0011010011',
 92: '111100000000',
 94: '001101111010',
 96: '00110111000',
 98: '001101110011',
 100: '001101111001',
 102: '111100000001',
 104: '1111000011000',
 106: '0011011100101',
 108: '1111000011011',
 110: '0011011110000',
 112: '11110000110100',
 114: '0011011110111',
 116: '0011011

In [11]:
huffman_encoded = ''.join([huff_dict[x] for x in gaps])
huffman_bytes = len(huffman_encoded) / 8
print(f'{len(huffman_encoded)} bits, {huffman_bytes} bytes, or {huffman_bytes / (2**20):0.3f} mb')

44712373 bits, 5589046.625 bytes, or 5.330 mb


# Achieve Target Compression Ratios

Your solution should correctly compute the following three compression ratios:

| Ratio       | Value              |
|-------------|--------------------|
| From fixed  | 36.125168653605158 |
| From binary |              86.03 |
| From ASCII  |              94.02 | 


## Double-Checking This

In [12]:
get_compression_ratio = lambda comp, uncomp: 100 * (uncomp - comp) / uncomp

print(f"From fixed:  {get_compression_ratio(huffman_bytes, gap_bytes)}")
print(f"From binary: {get_compression_ratio(huffman_bytes, prime_fixed_bytes):0.2f}")
print(f"From ASCII:  {get_compression_ratio(huffman_bytes, ascii_bytes):0.2f}")

From fixed:  36.12516865360516
From binary: 86.03
From ASCII:  94.02


# Above and Beyond

Here we wanted to make sure the Huffman encoding can be decoded, given the tree.

## Memory Hogs

In [13]:
# def decode(encoded, d, chunk = ''):
#     if encoded == '':
#         return []

#     if chunk in d.keys():
#         print(d[chunk], end=', ')
#         return [d[chunk]] + decode(encoded[1:], d, encoded[0])
#     else:
#         return decode(encoded[1:], d, chunk + encoded[0])
        

In [14]:
# def decode(encoded, d):
#   if encoded == []:
#     return []
#   else:
#     size = [len(x) for x in d.keys() if encoded[:len(x)] == x][0]
#     print(d[encoded[:size]], end=', ')
#     return [d[encoded[:size]]] + decode(encoded[size:], d)

In [15]:
# def decode(encoded, d):
#   while 1:
#     if encoded == []:
#       break
#     else:
#       size = [len(x) for x in d.keys() if encoded[:len(x)] == x][0]
#       print('\r' + str(len(encoded)), end='')
#       yield d[encoded[:size]]
#       encoded = encoded[size:]

## Working

In [16]:
# Davis did something similar with queues
# I'm still trying to find a way to do this
# functionally without crashing the runtime.
def decode(encoded, d):
  end = 1
  start = 0
  decoded = []
  while end <= len(encoded):
    test_key = encoded[start:end]
    if test_key in d:
      decoded.append(d[test_key])
      start = end
    end += 1
  return decoded

In [17]:
huff_dict_rev = {v: k for k, v in huff_dict.items()}
huff_decoded = decode(huffman_encoded, huff_dict_rev)
len(huff_decoded)

9999998

In [18]:
print(huff_decoded[:10])
print(huff_decoded[-10:])

[2, 2, 4, 2, 4, 2, 4, 6, 2, 6]
[18, 20, 6, 24, 10, 6, 12, 38, 4, 2]


In [19]:
gaps == huff_decoded

True

Good news! The Huffman encoding was decoded successfully.

# My Report on What I Did and What I Learned

## Fun


This exercise rekindled the interest in functional programming that I gained in DM1. Once I finished with the "Above and Beyond" portion, I started researching Haskell to develop my functional-programming skills.

## New

I learned that the fixed-length encoding of data could be more space-efficient than ASCII encoding, even for a relatively large amount of data. I also found that python does not return values of a list until the entire list is generated. This idea makes sense; however, it was frustrating when trying to decode the list of primes using the Huffman tree.

## Meaningful


Two things come to mind. First, I got more practice in functional programming. Because of this, I gained a renewed interest in functional programming. Second, I was able to implement a compression algorithm. I can see this being useful if I ever work with or create a proprietary file format.

## Other

I am also taking a networking class this semester. Because of this, one of the first things that came to mind was how sensitive Huffman encodings are to bit corruption. Let's say I was downloading a file that was compressed using Huffman encoding. If something were to happen to a single bit during the transfer process, then not only is that bit corrupt, but everything from that onward would most likely become corrupted.

Davis was the one who came up with most of the size calculations (he's the one who recognized that the ASCII file size would need to include newline characters). I mostly worked on the implementation of a Huffman tree in functional python. Paul and Brayden were able to come up with helpful questions and ideas. These helped in both understanding Huffman trees and debugging the implementation.

# What is True?
Click on each warranted checkbox to toggle it to True (or back to False). 

NOTE: *This only works in Colab. If you run it in some other Jupyter notebook client/server environment you may have to change False to True (or vice versa) manually.*

This self-assessment is subject to revision by a grader.

In [20]:
#@markdown ## What is True about what I did?
#@markdown ### I had fun.
cb00 = True #@param {type:'boolean'}
#@markdown ### I learned something new.
cb01 = True #@param {type:'boolean'}
#@markdown ### I achieved something meaningful, or something I can build upon at a later time.
cb02 = True #@param {type:'boolean'}
#@markdown ## What is True about my report?
#@markdown ### I wrote a sufficient number of well-written sentences.
cb03 = True #@param {type:'boolean'}
#@markdown ### My report is free of mechanical infelicities.
cb04 = True #@param {type:'boolean'}
#@markdown ### I used Grammarly (or something better described in my report) to check for MIs.
cb05 = True #@param {type:'boolean'}
#@markdown ### I reported on any connections I found between these problems and something I already know. 
cb06 = True #@param {type:'boolean'}
#@markdown ### I reported who were and what contribution each of my collaborators made.
cb07 = True #@param {type:'boolean'}
#@markdown ## What is True about my calculations?
#@markdown ### I correctly calculated the number of times each gap size occurs. 
cb08 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the number of bits per gap size with a fixed encoding.
cb09 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the Huffman encoding.
cb10 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the total number of bits encoded with the fixed encoding.
cb11 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from this fixed encoding.
cb12 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as 32-bit integer binary data.
cb13 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the binary size.
cb14 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the size of the first ten million primes encoded as ASCII data.
cb15 = True #@param {type:'boolean'}
#@markdown ### I correctly calculated the compression ratio from the ASCII size (just the primes, nothing else).
cb16 = True #@param {type:'boolean'}